In [ ]:
import pandas as pd

# TODO: update for state
STATE = 'ny'
YEARS = [str(yr) for yr in range(2013, 2020)]

In [ ]:
"""
File naming convension for ACS 5-year downloads:
state_year_[race|income].csv

"""

def get_filepath(state):
    return '../data/{state}/'.format(state=state)

def get_fname(state, year, dataset_type):
    return '{state}_{year}_{dataset_type}.csv'.format(
        state=state, year=year, dataset_type=dataset_type)

def get_filename(state, year, dataset_type):
    return get_filepath(state) + get_fname(state, year, dataset_type)


# geoid is the column we join data on
geoid_column_name = 'geoid'

geoid_column_map = {
    'GEO.id2': geoid_column_name,
    'GEO_ID': geoid_column_name
}

# for some reason particular file(s) have different column names- WHY?!
# filename_income_column_map = {
#     'ma_2017_income.csv': {
#         'HC03_EST_VC02': 'median income',
#         'HC03_MOE_VC02': 'median income margin of error',
#         'HD01_VD01': 'median income',
#         'HD02_VD01': 'median income margin of error'
#     }
# }
# Here is the default
income_column_map = {
    'HC02_EST_VC02': 'median income',
    'HC02_MOE_VC02': 'median income margin of error',
    'HD01_VD01': 'median income',
    'HD02_VD01': 'median income margin of error'
}

new_income_map = {
    'S1903_C03_001E': 'median income',
    'S1903_C03_001M': 'median income margin of error'
}

new_income_map_early = {
    'S1903_C02_001E': 'median income',
    'S1903_C02_001M': 'median income margin of error'
}

race_column_map = {
    'HD01_VD01': 'race: total households',
    'HD02_VD01': 'race: total households margin of error',
    'HD01_VD02': 'race: White',
    'HD01_VD03': 'race: Black',
    'HD01_VD05': 'race: Asian',
    'HD01_VD08': 'race: 2 or more races',
    # The following are combined into one value
    # 'HD01_VD04': 'race: American Indian and Alaska',
    # 'HD01_VD06': 'race: Native Hawaiian and Other',
    'HD01_VD07': 'race: Other',
}

new_race_column_map = {
    'B02001_001E': 'race: total households',
    'B02001_001M': 'race: total households margin of error',
    'B02001_002E': 'race: White',
    'B02001_003E': 'race: Black',
    'B02001_005E': 'race: Asian',
    'B02001_008E': 'race: 2 or more races',
    'B02001_007E': 'race: Other'
}

def race_combine_other(row):
    
    """Combines the values for the other races with american indian, hawaiian, etc"""
    try:
        return int(row['HD01_VD04']) + int(row['HD01_VD06']) + int(row['HD01_VD07'])
    except:
        return int(row['B02001_004E']) + int(row['B02001_006E']) + int(row['B02001_007E'])

def col_name_for_year(year, col_name):
    return str(year) + ' ' + col_name

def remove_labels(df):
    # drop the first row (the first row is a display label)
    df.drop([0], inplace=True)

def preprocess_df(df, year, column_map):
    # prune data
    # data from 2018-onwards only has GEO_ID not GEO.id2
    # if int(year) > 2017:
    #     for num in range(1, df.shape[0]+1):
    #         df.at[num, 'GEO_ID'] = df.at[num, 'GEO_ID'][-11:]
    for num in range(1, df.shape[0]+1):
        df.at[num, 'GEO_ID'] = df.at[num, 'GEO_ID'][-11:]  
    # rename columns
    column_name_map = {key: col_name_for_year(year, value) for key, value in column_map.items()}
    
    column_name_map.update(geoid_column_map)
    cols_to_drop = [col for col in df.columns if not col in column_name_map.keys()]

    df.drop(cols_to_drop,  axis=1,  inplace=True)
    df.rename(columns=column_name_map, inplace=True)
    df.set_index(geoid_column_name, inplace=True)
    return df


def get_race_df(state, year):
    filename = get_filename(STATE, year, 'race')
    df = pd.read_csv(filename)
    remove_labels(df)
    df['column'] = df.apply(race_combine_other, axis=1)
    map = new_race_column_map
    return preprocess_df(df, year, map)


def get_income_df(state, year):
    # so annoying that the columns change with files!
    if int(year) >= 2017:
        map = new_income_map
    else:
        map = new_income_map_early
    fname = get_fname(state, year, 'income')
    print('filename', fname)
    # if fname in filename_income_column_map:
    #     map = filename_income_column_map[fname]
    df = pd.read_csv(get_filename(STATE, year, 'income'))
    print('income:df', df)
    remove_labels(df)
    return preprocess_df(df, year, map)


In [ ]:
# We merge data into the income df

def add_df(df1, df2):
    # comebine the df's on geoid
    return pd.concat([df1, df2], axis=1, join='inner')

state_df = None
for year in YEARS:
    print('handling files for year ', year)
    income_df = get_income_df(STATE, year)
    if state_df is None:
        state_df = income_df
    else:
        state_df = add_df(state_df, income_df)
    # print(state_df)
    race_df = get_race_df(STATE, year)
    state_df = add_df(state_df, race_df)
# state_df = state_df.loc[state_df[geoid_column_name] == '48201410802']
state_df.head()



In [ ]:
# Save the giant dataframe to CSV
output_csvfilename = get_filepath(STATE) + 'race_and_income_data.csv'
state_df.to_csv(output_csvfilename)
print('saved data to ',  output_csvfilename)

In [ ]:
import geopandas as gpd

In [ ]:
"""Creates shapefile from NYC open data download.
This shapefile does not have geoids, so must add them based on its other data.

"""

NY_STATE_CODE = '36'

nyc_shapefile_attributes = {
    geoid_column_name: geoid_column_name,
    'ntaname': 'Name', # name of neighborhood in this dataset
    # Tracts are duplicated across boros.
    # The unique key is 'boro_ct201' which is the concatenation of boro id and ct
    # 'ct2010': 'tract',
    'shape_area': 'shape_area',
    'shape_leng': 'shape_leng',
    'geometry': 'geometry'
}
ma_shapefile_attributes = {
    'GEOID10': geoid_column_name,
    'NAMELSAD10': 'Name', # 'Census Tract ###'  in this dataset
    'SHAPE_AREA': 'shape_area',
    'SHAPE_LEN': 'shape_leng',
    'geometry': 'geometry'
}

tiger_shapefile_attributes = {
    'GEOID': geoid_column_name,
    'NAMELSAD': 'Name',
    'AREA': 'shape_area',
    'geometry': 'geometry'
}

# Mapping of boro names to county code for geoid
# Taken from wikipedia info: https://en.wikipedia.org/wiki/List_of_counties_in_New_York
nyc_boro_to_county_code = {
    'Bronx':'005',
    'Queens':'081',
    'Brooklyn':'047',
    'Manhattan':'061',
    'Staten Island':'085'
}


def get_nyc_shapefile():
    shapefile_filename = get_filepath('ny') + 'city_census_tracts_shapefile/geo_export_6f3df1e4-1be2-4395-ba6c-3e15b0a10221.shp'
    shapefile_df = gpd.read_file(shapefile_filename)
    shapefile_df[geoid_column_name] = shapefile_df.apply(get_nyc_geoid, axis=1)
    return shapefile_df


def get_nyc_county_code(row):
    boro_name = row['boro_name']
    return nyc_boro_to_county_code[boro_name]


def get_nyc_geoid(row):
    state_code = NY_STATE_CODE
    county_code = get_nyc_county_code(row)
    tract_code = row['ct2010']
    return str(state_code) + str(county_code) + str(tract_code)

shapefile_df = None
if STATE == 'ny':
    shapefile_df = get_nyc_shapefile()
    shapefile_attributes = nyc_shapefile_attributes
elif STATE == 'ma':
    shapefile_filename = get_filepath(STATE) + 'shapefile/boston-brookline-cambridge-somerville-everett.shp'
    shapefile_df = gpd.read_file(shapefile_filename)
    shapefile_attributes = ma_shapefile_attributes
elif STATE == 'dc':
    shapefile_filename = get_filepath(STATE) + 'shapefile/DC/tl_2019_11_tract.shp'
    dc_shapefile_df = gpd.read_file(shapefile_filename)

    shapefile_filename = get_filepath(STATE) + 'shapefile/MD/tl_2019_24_tract.shp'
    md_shapefile_df = gpd.read_file(shapefile_filename)
    md_shapefile_df1 = md_shapefile_df.loc[md_shapefile_df["COUNTYFP"] == '031']
    md_shapefile_df2 = md_shapefile_df.loc[md_shapefile_df["COUNTYFP"] == '033']
    md_shapefile_df = pd.concat([md_shapefile_df1, md_shapefile_df2], axis = 0)

    shapefile_filename = get_filepath(STATE) + 'shapefile/VA/tl_2019_51_tract.shp'
    va_shapefile_df = gpd.read_file(shapefile_filename)
    va_shapefile_df1 = va_shapefile_df.loc[va_shapefile_df["COUNTYFP"] == '013']
    va_shapefile_df2 = va_shapefile_df.loc[va_shapefile_df["COUNTYFP"] == '510']
    va_shapefile_df = pd.concat([va_shapefile_df1, va_shapefile_df2], axis = 0)

    shapefile_df = pd.concat([dc_shapefile_df, va_shapefile_df, md_shapefile_df], axis = 0)
    shapefile_df["AREA"] = shapefile_df["ALAND"].astype(int) + shapefile_df["AWATER"].astype(int)
    shapefile_attributes = tiger_shapefile_attributes

elif STATE == 'il':
    shapefile_filename = get_filepath(STATE) + 'shapefile/tl_2019_17_tract.shp'
    shapefile_df = gpd.read_file(shapefile_filename)
    shapefile_df = shapefile_df.loc[shapefile_df["COUNTYFP"] == '031']
    shapefile_df["AREA"] = shapefile_df["ALAND"].astype(int) + shapefile_df["AWATER"].astype(int)
    shapefile_attributes = tiger_shapefile_attributes
elif STATE == 'pa':
    shapefile_filename = get_filepath(STATE) + 'shapefiles/tl_2019_42_tract.shp'
    shapefile_df = gpd.read_file(shapefile_filename)
    shapefile_df = shapefile_df.loc[shapefile_df["COUNTYFP"] == '101']
    shapefile_df["AREA"] = shapefile_df["ALAND"].astype(int) + shapefile_df["AWATER"].astype(int)
    shapefile_attributes = tiger_shapefile_attributes

shapefile_df


In [ ]:
# Map column names and remove columns
shapefile_cols_to_drop = [col for col in shapefile_df.columns if not col in shapefile_attributes.keys()]
shapefile_df.drop(shapefile_cols_to_drop,  axis=1,  inplace=True)
shapefile_df.rename(columns=shapefile_attributes, inplace=True)
shapefile_df.head()
shapefile_df.set_index(geoid_column_name, inplace=True)
shapefile_df.head() 



In [ ]:
# Want projection CRS84 which is equivalent to EPSG:4326
shapefile_df = shapefile_df.to_crs({'init': 'epsg:4326'})

%matplotlib inline
shapefile_df.plot()

In [ ]:
merged_shapes = shapefile_df.merge(state_df, on=geoid_column_name)
print('shape', merged_shapes.shape)
merged_shapes.head()

In [ ]:
from shapely.geometry.polygon import Polygon
from shapely.geometry.multipolygon import MultiPolygon

In [ ]:
to_filename = get_filepath(STATE) + STATE  + '_census_tracts.geojson'
print('saving to ',to_filename)
merged_shapes["geometry"] = [MultiPolygon([feature]) if type(feature) == Polygon \
    else feature for feature in merged_shapes["geometry"]]
merged_shapes.to_file(to_filename, driver='GeoJSON')
print('saved')
sort = False